In [1]:
import imgmgmt as img
from dft import fft, phaseIFFT, ampIFFT, oneDomainIFFT, np
from pgmf import openImg, saveContrastImg, saveNormImg
from convolution import convolution, filterFreqDomain, idealLPfilter, gaussianLPfilter, medianfilter, rms

In [2]:
depth, cross = openImg('./1/Cross.pgm')
cross = img.inverse(cross)
img.padding(cross, 56)
cross = img.inverse(cross)

In [3]:
# 1.1
crossFT = fft(cross)
cross_amplitude, cross_phase = img.extractAmplitudeAndPhase(crossFT)
saveContrastImg('./1/CrossPhase.pgm', cross_phase, 255)
saveNormImg('./1/CrossAmplitude.pgm', cross_amplitude, 255)

In [4]:
# 1.2
shiftIFT = oneDomainIFFT(img.shift(crossFT, 20, 30))
saveContrastImg('./1/Cross_Shift.pgm', shiftIFT, 255)

In [5]:
# 1.3
rotated_img = img.rotate(cross, 30, 255)
saveContrastImg("./1/Cross_rotate.pgm", rotated_img, 255)
rotateFT = fft(rotated_img)
rotate_amplitude, rotate_phase = img.extractAmplitudeAndPhase(rotateFT)
saveContrastImg('./1/Cross_rotate_phase.pgm', rotate_phase, 255)
saveNormImg('./1/Cross_rotate_amplitude.pgm', rotate_amplitude, 255)

In [6]:
# 1.4
downsampling = img.downScale(cross, 100, 100)
downFT = img.inverse(downsampling)
img.padding(downFT,28)
downFT = img.inverse(downFT)

downFT = fft(downFT)
down_amp, down_phase = img.extractAmplitudeAndPhase(downFT)

saveNormImg('./1/Cross_Down_Sampling_Amp.pgm', down_amp, 255)
saveContrastImg('./1/Cross_Down_Sampling_Phase.pgm', down_phase, 255)

In [7]:
# 1.5
cross_ifft_phase = phaseIFFT(cross_phase)
cross_ifft_amp = ampIFFT(cross_amplitude)

saveContrastImg('./1/Cross_ifft_phase.pgm', cross_ifft_phase, 255)
saveContrastImg('./1/Cross_ifft_amplitude.pgm', cross_ifft_amp, 255)

In [8]:
# 1.6
depth, lenna = openImg('./1/Lenna.pgm')

lennaFT = fft(lenna)
lenna_amplitude, lenna_phase = img.extractAmplitudeAndPhase(lennaFT)

lenna_ifft_amp = ampIFFT(lenna_amplitude)
lenna_ifft_phase = phaseIFFT(lenna_phase)

saveNormImg('./1/LennaAmplitude.pgm', lenna_amplitude, 255)
saveContrastImg('./1/Lenna_ifft_phase.pgm', lenna_ifft_phase, 255)

In [9]:
# 1.7
kernel= np.array([[1, 1, 1]]*3) * (1/9)
depth, chess = openImg('./1/Chess.pgm')

chessFT = fft(chess)

kernel_pad = [row.copy() for row in kernel]
ker_pad = np.delete(np.delete(np.pad(kernel_pad, 127), 0, 1), 0, 0)
chess_filter = filterFreqDomain(chessFT, fft(ker_pad))
# img.padding(chess, 2)
chess_convolute = convolution(np.pad(chess, 1), np.array(kernel))

filterIFT = oneDomainIFFT(chess_filter)

saveContrastImg('./1/Chess_kernel_blur.pgm', chess_convolute, depth)
saveContrastImg('./1/Chess_filter_freq.pgm', filterIFT, depth)

In [10]:
# 2.1
depth, cross = openImg('./1/Cross.pgm')

crossFT = fft(cross)
cross_ilp30 = idealLPfilter(crossFT, 30)
cross_ilp30_ift = oneDomainIFFT(cross_ilp30)
cross_ilp50 = idealLPfilter(crossFT, 50)
cross_ilp50_ift = oneDomainIFFT(cross_ilp50)
cross_ilp100 = idealLPfilter(crossFT, 100)
cross_ilp100_ift = oneDomainIFFT(cross_ilp100)

saveContrastImg('./2/ideal_lp_30.pgm', cross_ilp30_ift, depth)
saveContrastImg('./2/ideal_lp_50.pgm', cross_ilp50_ift, depth)
saveContrastImg('./2/ideal_lp_100.pgm', cross_ilp100_ift, depth)

cross_glp30 = gaussianLPfilter(crossFT, 30)
cross_glp30_ift = oneDomainIFFT(cross_glp30)
cross_glp50 = gaussianLPfilter(crossFT, 50)
cross_glp50_ift = oneDomainIFFT(cross_glp50)
cross_glp100 = gaussianLPfilter(crossFT, 100)
cross_glp100_ift = oneDomainIFFT(cross_glp100)

saveContrastImg('./2/gaussian_lp_30.pgm', cross_glp30_ift, depth)
saveContrastImg('./2/gaussian_lp_50.pgm', cross_glp50_ift, depth)
saveContrastImg('./2/gaussian_lp_100.pgm', cross_glp100_ift, depth)

In [18]:
# 2.2
depth, chess_noise = openImg('./2/Chess_noise.pgm')
cnFT = fft(chess_noise)

chess_noise_glp = oneDomainIFFT(gaussianLPfilter(cnFT, 50))
chess_noise_ilp = oneDomainIFFT(idealLPfilter(cnFT, 50))
chess_noise_med = medianfilter(chess_noise)

saveContrastImg('./2/Chess_noise_gaussian.pgm', chess_noise_glp, depth)
saveContrastImg('./2/Chess_noise_ideal.pgm', chess_noise_ilp, depth)
saveContrastImg('./2/Chess_noise_median.pgm', chess_noise_med, depth)

chess_noise = np.array(chess_noise)
print('rms(chess, chess_noise): {:.5f}'.format(rms(np.array(chess), chess_noise)))
print('rms(chess, glp50):       {:.5f}'.format(rms(np.array(chess), np.array(chess_noise_glp))))
print('rms(chess, ilp50):       {:.5f}'.format(rms(np.array(chess), np.array(chess_noise_ilp))))
print('rms(chess, med):         {:.5f}\n'.format(rms(np.array(chess), np.array(chess_noise_med))))

depth, lenna_noise = openImg('./2/Lenna_noise.pgm')
lnFT = fft(lenna_noise)

lenna_noise_glp = oneDomainIFFT(gaussianLPfilter(lnFT, 50))
lenna_noise_ilp = oneDomainIFFT(idealLPfilter(lnFT, 50))
lenna_noise_med = medianfilter(lenna_noise)

saveContrastImg('./2/Lenna_noise_gaussian.pgm', lenna_noise_glp, depth)
saveContrastImg('./2/Lenna_noise_ideal.pgm', lenna_noise_ilp, depth)
saveContrastImg('./2/Lenna_noise_median.pgm', lenna_noise_med, depth)

lenna_noise = np.array(lenna_noise)
print('rms(lenna, lenna_noise): {:.5f}'.format(rms(np.array(lenna), lenna_noise)))
print('rms(lenna, glp50):       {:.5f}'.format(rms(np.array(lenna), np.array(lenna_noise_glp))))
print('rms(lenna, ilp50):       {:.5f}'.format(rms(np.array(lenna), np.array(lenna_noise_ilp))))
print('rms(lenna, med):         {:.5f}'.format(rms(np.array(lenna), np.array(lenna_noise_med))))

rms(chess, chess_noise): 24.58521
rms(chess, glp50):       14.08603
rms(chess, ilp50):       16.85038
rms(chess, med):         17.24524

rms(lenna, lenna_noise): 25.28078
rms(lenna, glp50):       11.21319
rms(lenna, ilp50):       12.68227
rms(lenna, med):         12.62758
